#### Install required package

In [1]:
# pip install pymongo


#### Create a pymongo connection

In [ ]:
import os
import pandas as pd
pd.set_option("display.max_column", None)

from pymongo import MongoClient
from bson import json_util
import json

mongo_url = input("Enter the Mongo connection string: ")
client = MongoClient(mongo_url)
print("Python-MongoDB connection created succesfully -\n", client)


Python-MongoDB connection created succesfully -
 MongoClient(host=['3.146.176.180:27017'], document_class=dict, tz_aware=False, connect=True)


#### View list of collections in the database

In [3]:
db = client['talentsync-backend-production']
collection_names = db.list_collection_names()
print(collection_names)


['candidate_questions', 'job_hr_questions', 'candidates', 'auth_keys', 'jobs', 'audit_logs', 'job_hr_settings', 'batches', 'questions', 'candidates_errors', 'users', 'companies', 'messages', 'auth_tokens']


#### Find all the documents in this particular collection

In [4]:
collection = db['companies']
results = collection.find()
print(results)


In [5]:
collection = db['companies']
results = collection.find()
for doc in results:
    print(doc)


{'_id': ObjectId('681c861b1cf44d32fe03095e'), 'name': 'Klizo Solutions', 'organization_size': '11-50', 'company_type': 'corporation', 'created_at': datetime.datetime(2025, 5, 8, 10, 23, 23, 615000), 'updated_at': datetime.datetime(2025, 5, 8, 10, 23, 23, 615000), 'is_deleted': False}
{'_id': ObjectId('681c86b51cf44d32fe030963'), 'name': 'Vietnam Australia International School', 'organization_size': '200+', 'company_type': 'corporation', 'created_at': datetime.datetime(2025, 5, 8, 10, 25, 57, 981000), 'updated_at': datetime.datetime(2025, 5, 8, 10, 25, 57, 981000), 'is_deleted': False}
{'_id': ObjectId('681c872f1cf44d32fe03096d'), 'name': 'Esco Beach', 'organization_size': '51-200', 'company_type': 'corporation', 'created_at': datetime.datetime(2025, 5, 8, 10, 27, 59, 721000), 'updated_at': datetime.datetime(2025, 5, 8, 10, 27, 59, 721000), 'is_deleted': False}
{'_id': ObjectId('682200046342bdaa11723975'), 'name': 'Furarma Resort Danang', 'organization_size': '200+', 'company_type': 'co

#### View the first document in this collection

In [6]:
collection = db['companies']
result = collection.find_one()
print(result)


{'_id': ObjectId('681c861b1cf44d32fe03095e'), 'name': 'Klizo Solutions', 'organization_size': '11-50', 'company_type': 'corporation', 'created_at': datetime.datetime(2025, 5, 8, 10, 23, 23, 615000), 'updated_at': datetime.datetime(2025, 5, 8, 10, 23, 23, 615000), 'is_deleted': False}


In [7]:
collection = db['companies']
result = collection.find_one()
print(json.dumps(result, indent=4, default=json_util.default))


{
    "_id": {
        "$oid": "681c861b1cf44d32fe03095e"
    },
    "name": "Klizo Solutions",
    "organization_size": "11-50",
    "company_type": "corporation",
    "created_at": {
        "$date": "2025-05-08T10:23:23.615Z"
    },
    "updated_at": {
        "$date": "2025-05-08T10:23:23.615Z"
    },
    "is_deleted": false
}


#### How many documents are listed in this collection

In [8]:
# print("shape: ", df.shape)


In [9]:
num_rows = collection.count_documents({})
print("shape:", num_rows )


shape: 38


#### How many unique fields are there in this collection

In [10]:
# print("Columns: ", df.columns)


In [11]:
all_fields = set()
for field in collection.find():
    all_fields.update(field.keys())
    
print("Columns:", list(all_fields))


Columns: ['updated_at', 'company_type', 'organization_size', '_id', 'name', 'created_at', 'is_deleted']


#### Value counts for seperate fields

##### 'is_deleted'

In [12]:
# df['is_deleted'].value_counts()


In [13]:
query = [
    {
        "$group": {
            "_id": "$is_deleted",
            "count": {"$sum": 1}
        }
    }
]

result = collection.aggregate(query)

for doc in result:
    print(f"{doc['_id']}: {doc['count']}")


False: 38


##### 'organization_size'

In [14]:
query = [
    {
        "$group": {
            "_id": "$organization_size",
            "count": {"$sum": 1}
        }
    }
]

result = collection.aggregate(query)

for doc in result:
    print(f"{doc['_id']}: {doc['count']}")


51-200: 10
200+: 11
1-10: 9
11-50: 8


##### 'company_type'

In [15]:
query = [
    {
        "$group":{
            "_id": "$company_type",
            "count": {"$sum": 1}
        }
    }
]

result = collection.aggregate(query)

for doc in result:
    print(f"{doc['_id']}: {doc['count']}")


startup: 8
consulting: 1
small-business: 4
ecommerce: 2
non-profit: 1
enterprise: 6
corporation: 10
manufacturing: 5
freelance: 1


#### Count of unique companies

In [16]:
# df[['name']].count()


In [17]:
query = [
    {
        "$match":{
            "name": {"$exists": True, "$ne": None}
        }
    },
    {
        "$count": "company_count"
    }
]

result = list(collection.aggregate(query))
print(result)


[{'company_count': 38}]


In [18]:
query = [
    {
        "$match":{
            "name": {"$exists": True, "$ne": None}
        }
    },
    {
        "$count": "company_count"
    }
]

result = list(collection.aggregate(query))
print(result[0])


{'company_count': 38}


In [19]:
query = [
    {
        "$match": {
            "name": { "$exists": True, "$ne": None }
        }
    },
    {
        "$count": "name_count"
    }
]

result = list(collection.aggregate(query))
print(result[0]['name_count'] if result else 0)


38


#### View the Unique Companies' names

In [20]:
# df[['name']]


In [21]:
query = {"name":1}
results = collection.find({}, query)
for doc in results:
    print(doc)


{'_id': ObjectId('681c861b1cf44d32fe03095e'), 'name': 'Klizo Solutions'}
{'_id': ObjectId('681c86b51cf44d32fe030963'), 'name': 'Vietnam Australia International School'}
{'_id': ObjectId('681c872f1cf44d32fe03096d'), 'name': 'Esco Beach'}
{'_id': ObjectId('682200046342bdaa11723975'), 'name': 'Furarma Resort Danang'}
{'_id': ObjectId('682200936342bdaa1172397a'), 'name': 'Furama Resort Da Nang'}
{'_id': ObjectId('6822e3346342bdaa117239ba'), 'name': 'Singapore Internation School'}
{'_id': ObjectId('68243b106342bdaa11723a5d'), 'name': 'Production Company'}
{'_id': ObjectId('682472001b85c2b6f71a5ba5'), 'name': 'Demo Company'}
{'_id': ObjectId('682475961b85c2b6f71a5baa'), 'name': 'QA_Company'}
{'_id': ObjectId('68259b511b85c2b6f71a5bdf'), 'name': 'Bunny House'}
{'_id': ObjectId('68307b590582447f62edadd8'), 'name': 'Klizo QA'}
{'_id': ObjectId('6835e66bc165a9e593e1ebb1'), 'name': 'ESCO Beach Vietnam'}
{'_id': ObjectId('683dc273b72469b8444e42e0'), 'name': 'Show Me'}
{'_id': ObjectId('683ee19fb72

In [22]:
query = {"name": 1, "_id": 0}
results = collection.find({}, query)
for doc in results:
    print(doc)


{'name': 'Klizo Solutions'}
{'name': 'Vietnam Australia International School'}
{'name': 'Esco Beach'}
{'name': 'Furarma Resort Danang'}
{'name': 'Furama Resort Da Nang'}
{'name': 'Singapore Internation School'}
{'name': 'Production Company'}
{'name': 'Demo Company'}
{'name': 'QA_Company'}
{'name': 'Bunny House'}
{'name': 'Klizo QA'}
{'name': 'ESCO Beach Vietnam'}
{'name': 'Show Me'}
{'name': 'F comany'}
{'name': 'Test'}
{'name': 'Apollo English Vietnam'}
{'name': 'Demo Company 2'}
{'name': 'Quality Analyst Company'}
{'name': 'In magna dolore temp'}
{'name': 'Demo Company 3'}
{'name': 'Demo Company 4'}
{'name': 'Ad officia sed velit'}
{'name': 'Einstein Tech'}
{'name': 'New Company Names'}
{'name': 'Testuikji87k6j7i'}
{'name': 'Klizo Solutions2'}
{'name': 'Klizo'}
{'name': 'Test Company'}
{'name': 'Testing'}
{'name': "Rooj's Company"}
{'name': 'Veniam at illum vo'}
{'name': 'internal-company'}
{'name': "Rooj's Solutions"}
{'name': 'Interview Screener_Klizo Solution'}
{'name': 'ABC Compa

#### Filter out fields base on defined conditions

##### Filter company names based on mentioned organization_size

In [23]:
# filtered_df = df[df['organization_size'] == '11-50'][['name']]


In [24]:
filter_expression = {"organization_size": "11-50"}
projection = {"name": 1, "_id": 0}

results = collection.find(filter_expression, projection)

for doc in results:
    print(doc)


{'name': 'Klizo Solutions'}
{'name': 'Demo Company'}
{'name': 'Demo Company 2'}
{'name': 'Ad officia sed velit'}
{'name': 'New Company Names'}
{'name': 'Veniam at illum vo'}
{'name': 'ABC Company'}
{'name': 'Abcd Company'}


In [25]:
cursor = collection.find(
    {"organization_size": "11-50"},  
    {"name": 1, "_id": 0}            
)

for doc in cursor:
    print(doc)


{'name': 'Klizo Solutions'}
{'name': 'Demo Company'}
{'name': 'Demo Company 2'}
{'name': 'Ad officia sed velit'}
{'name': 'New Company Names'}
{'name': 'Veniam at illum vo'}
{'name': 'ABC Company'}
{'name': 'Abcd Company'}


In [26]:
query = [
    {
        "$match": {
            "organization_size": "11-50",
            "name": { "$exists": True, "$ne": None }
        }
    },
    {
        "$project": {
            "_id": 0,
            "name": 1
        }
    }
]

result = list(collection.aggregate(query))
for doc in result:
    print(doc)


{'name': 'Klizo Solutions'}
{'name': 'Demo Company'}
{'name': 'Demo Company 2'}
{'name': 'Ad officia sed velit'}
{'name': 'New Company Names'}
{'name': 'Veniam at illum vo'}
{'name': 'ABC Company'}
{'name': 'Abcd Company'}


##### Filter company names and company_type based on mentioned organization_size

In [27]:
# filtered_df = df[df['organization_size'] == '11-50'][['name', 'company_type']]


In [28]:
query = [
    {
        "$match": {
            "organization_size": "11-50",
            "name": { "$exists": True, "$ne": None }
        }
    },
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "company_type": 1
        }
    }
]

result = list(collection.aggregate(query))
for doc in result:
    print(doc)


{'name': 'Klizo Solutions', 'company_type': 'corporation'}
{'name': 'Demo Company', 'company_type': 'small-business'}
{'name': 'Demo Company 2', 'company_type': 'corporation'}
{'name': 'Ad officia sed velit', 'company_type': 'corporation'}
{'name': 'New Company Names', 'company_type': 'enterprise'}
{'name': 'Veniam at illum vo', 'company_type': 'manufacturing'}
{'name': 'ABC Company', 'company_type': 'corporation'}
{'name': 'Abcd Company', 'company_type': 'enterprise'}


#### Sorting company name in descending order then group by organization_size

In [29]:
# SELECT organization_size, name FROM df GROUP BY organization_size ORDER BY name DESC

# df_sorted = df.sort_values(by='name', ascending=False)
# df_grouped = df_sorted.groupby('organization_size').first().reset_index()
# df_result = df_grouped[['organization_size', 'name']]


##### return only the top company name for each group of organization size

In [30]:
query = [
    {
        "$sort": {
            "name": -1  
        }
    },
    {
        "$group": {
            "_id": "$organization_size",
            "name": { "$first": "$name" }
        }
    },
    {
        "$project": {
            "organization_size": "$_id",
            "name": 1,
            "_id": 0
        }
    }
]

result = list(collection.aggregate(query))
for doc in result:
    print(doc)


{'name': 'Vietnam Australia International School', 'organization_size': '200+'}
{'name': 'Quality Analyst Company', 'organization_size': '51-200'}
{'name': 'internal-company', 'organization_size': '1-10'}
{'name': 'Veniam at illum vo', 'organization_size': '11-50'}


##### return all the company name for each group of organization size

In [31]:
# df_result = df[['organization_size', 'name']].sort_values(by='organization_size', ascending=False)


In [32]:
query = [
    {
        "$project": {
            "organization_size": 1,
            "name": 1,
            "_id": 0
        }
    },
    {
        "$sort": {
            "organization_size": -1 
        }
    }
]

result = list(collection.aggregate(query))
for doc in result:
    print(doc)


{'name': 'Klizo Solutions2', 'organization_size': '51-200'}
{'name': 'Esco Beach', 'organization_size': '51-200'}
{'name': 'Interview Screener_Klizo Solution', 'organization_size': '51-200'}
{'name': 'In magna dolore temp', 'organization_size': '51-200'}
{'name': 'Production Company', 'organization_size': '51-200'}
{'name': 'Quality Analyst Company', 'organization_size': '51-200'}
{'name': 'QA_Company', 'organization_size': '51-200'}
{'name': 'Klizo QA', 'organization_size': '51-200'}
{'name': 'ESCO Beach Vietnam', 'organization_size': '51-200'}
{'name': 'F comany', 'organization_size': '51-200'}
{'name': 'Testuikji87k6j7i', 'organization_size': '200+'}
{'name': 'Test Company', 'organization_size': '200+'}
{'name': 'Testing', 'organization_size': '200+'}
{'name': 'Apollo English Vietnam', 'organization_size': '200+'}
{'name': 'Test', 'organization_size': '200+'}
{'name': 'Show Me', 'organization_size': '200+'}
{'name': "Rooj's Company", 'organization_size': '200+'}
{'name': 'Singapore 